# Разработчик Ангел Давид Кантор

In [2]:
import sys
import subprocess


subprocess.run([sys.executable, "-m", "pip", "install", "pandas", "numpy", "scikit-learn"])

print("Instalación completada. Ahora importa las librerías:")
import pandas as pd
import numpy as np
print(f"Pandas {pd.__version__} y NumPy {np.__version__} instalados correctamente")

Instalación completada. Ahora importa las librerías:
Pandas 2.3.3 y NumPy 2.3.5 instalados correctamente


In [3]:
%%capture
!curl -L -o positive.csv https://raw.githubusercontent.com/Gavroshe/RuTweetCorp/master/positive.csv
!curl -L -o negative.csv https://raw.githubusercontent.com/Gavroshe/RuTweetCorp/master/negative.csv

In [4]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [5]:
import os
import pandas as pd

# Verifica el directorio actual
print("Directorio actual:", os.getcwd())
print("Archivos aquí:", os.listdir('.'))



positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)


negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative]) #This united the function -

Directorio actual: c:\Users\Usuario\Videos\Innovation Web\Project\analis_emotion\src
Archivos aquí: ['Emotions.ipynb', 'negative.csv', 'positive.csv']


In [6]:
print(df.head())

                                                text     label
0  @first_timee хоть я и школота, но поверь, у на...  positive
1  Да, все-таки он немного похож на него. Но мой ...  positive
2  RT @KatiaCheh: Ну ты идиотка) я испугалась за ...  positive
3  RT @digger2912: "Кто то в углу сидит и погибае...  positive
4  @irina_dyshkant Вот что значит страшилка :D\nН...  positive


In [7]:
df.sample(5, random_state=40)

,text,label
15931,RT @Blawar_1337: Теперь у нас с @Wake_UA появи...,positive
59532,с днём рождения зайка*))) ухх погуляем мы сего...,positive
47185,RT @Shumkova0406199: @ann_safina Вов вов вов А...,negative
42002,"Надо выдернуть звуковую дорожку из ""Доктора Ка...",positive
109035,@_hassliebe_ может все таки на этой неделе вер...,negative


# ПРЕДОБРАБОТКА

We divide the data into training and test samples using the function - Разбиваем данные на обучающую и тесторовую выборки с помощью функции

In [46]:
x_train, x_test, y_train, y_test = train_test_split(
    df.text, df.label,
    test_size=0.25,
    random_state=42,
    
    )


print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(170125,) (56709,) (170125,) (56709,)


In [9]:
y_train[:5]

36316    negative
84886    negative
54712    positive
5832     positive
21157    negative
Name: label, dtype: object

In [10]:
x_train[:5]

36316    Съездила с его родителями на пересыльный..Каки...
84886    @NastyaKarandeev я гуляла на улице весь вечер,...
54712    @cathriene_17 сверху ты - твои мечты. Сверху я...
5832     @Tatomkeen а я только вернулась из литвы и впе...
21157    я жирафа! ну как можно было удалить то, что я ...
Name: text, dtype: object

In [11]:
y_train.value_counts()

label
positive    86141
negative    83984
Name: count, dtype: int64

In [12]:
y_test[:5]

36155     positive
41115     negative
85868     positive
104611    positive
37423     negative
Name: label, dtype: object

#Baseline: classification of unprocessed n-grams
We will try to get the transformation of sentences into a numerical vector.
Мы попробуем получить преобразование предложений в численный вектор.


In [13]:
!pip3 install nltk

In [14]:
import subprocess
import sys

# Instalar usando Python directamente (sin rutas con espacios)
result = subprocess.run([sys.executable, "-m", "pip", "install", "nltk"], 
                       capture_output=True, text=True)

if result.returncode == 0:
    print("✅ NLTK instalado correctamente")
else:
    print("⚠️  Intentando método alternativo...")
    # Método alternativo
    import pip
    pip.main(['install', 'nltk'])

✅ NLTK instalado correctamente


Функция для работы с n-граммами реализована в библиотке nltk (Natural Language ToolKit), импортируем эту функцию:

In [15]:
import nltk
from nltk import ngrams


In [16]:
sentence = 'Если б мне платили каждый раз'.split()
sentence

['Если', 'б', 'мне', 'платили', 'каждый', 'раз']

Чтобы получить n-грамму для такой последовательности, используем функцию ngrams().

In [17]:
list(ngrams(sentence, 1))#униграммы

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

In [18]:
list(ngrams(sentence, 2))#биграммы

[('Если', 'б'),
 ('б', 'мне'),
 ('мне', 'платили'),
 ('платили', 'каждый'),
 ('каждый', 'раз')]

In [19]:
list(ngrams(sentence, 3))#триграммы

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

# Векторизация - Vectorization

In [20]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее

Настраиваем векторизатор - We configured the vectorizer

In [49]:
#Метод Bow - Мешок Слов
vectorizer = CountVectorizer(ngram_range=(1,1))

In [50]:
vectorized_x_train = vectorizer.fit_transform(x_train)

In [51]:
list(vectorizer.vocabulary_.items())[:10]

[('хватит', 232913),
 ('писать', 182291),
 ('такие', 219481),
 ('глупости', 119889),
 ('надо', 162235),
 ('правильно', 191630),
 ('речь', 203440),
 ('формулировать', 231251),
 ('отдохнуть', 175540),
 ('от', 175056)]

После инициализации vectorizer можно обучить на наших данных.

В нашей выборке 170125 текстов (твитов), в них встречается 243835 разных слов.

In [52]:
vectorized_x_train.shape

(170125, 243585)

In [53]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(vectorized_x_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [54]:
vectorized_x_test= vectorizer.transform(x_test)
#print(vectorized_x_test)

In [55]:
pred1 = clf.predict(vectorized_x_test)

print(classification_report(y_test, pred1))

              precision    recall  f1-score   support

    negative       0.76      0.77      0.77     28108
    positive       0.77      0.77      0.77     28601

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



# TF-IDF Векторизация 

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# инициализируем векторизатор, в качестве переменных используем униграммы
tfidfvectorizer = TfidfVectorizer(ngram_range=(1,5))
# обучаем его и сразу применяем к x_train
tfidf_vectorized_x_train = tfidfvectorizer.fit_transform(x_train)
# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42, max_iter=1000)
# применяем обученный векторизатор к тестовым данным
clf.fit(tfidf_vectorized_x_train, y_train)

# получаем предсказания и выводим информацию о качестве
tfidf_vectorized_x_test  = tfidfvectorizer.transform(x_test)

# # получаем предсказания и выводим информацию о качестве
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.73      0.78      0.75     27939
    positive       0.77      0.72      0.74     28770

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709



# TF-IDF Векторизация Биграмма

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# инициализируем векторизатор, в качестве переменных используем униграммы
tfidfvectorizer2 = TfidfVectorizer(ngram_range=(2,2))
# обучаем его и сразу применяем к x_train
tfidf_vectorized_x_train_2 = tfidfvectorizer2.fit_transform(x_train)
# инициализируем и обучаем классификатор
clf2 = LogisticRegression(random_state=42, max_iter=1000)
# применяем обученный векторизатор к тестовым данным
clf.fit(tfidf_vectorized_x_train_2, y_train)

# получаем предсказания и выводим информацию о качестве
tfidf_vectorized_x_test_2  = tfidfvectorizer2.transform(x_test)

# # получаем предсказания и выводим информацию о качестве
pred2 = clf.predict(tfidf_vectorized_x_test_2)
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

    negative       0.72      0.67      0.69     27939
    positive       0.70      0.75      0.73     28770

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709



# TF-IDF Векторизация Триграмма

In [28]:
# инициализируем векторизатор, в качестве переменных используем униграммы
tfidfvectorizer3 = TfidfVectorizer(ngram_range=(3,3))
# обучаем его и сразу применяем к x_train
tfidf_vectorized_x_train_3 = tfidfvectorizer3.fit_transform(x_train)
# инициализируем и обучаем классификатор
clf3 = LogisticRegression(random_state=42, max_iter=1000)
# применяем обученный векторизатор к тестовым данным
clf.fit(tfidf_vectorized_x_train_3, y_train)

# получаем предсказания и выводим информацию о качестве
tfidf_vectorized_x_test_3  = tfidfvectorizer3.transform(x_test)

# # получаем предсказания и выводим информацию о качестве
pred3 = clf.predict(tfidf_vectorized_x_test_3)
print(classification_report(y_test, pred3))

              precision    recall  f1-score   support

    negative       0.73      0.46      0.56     27939
    positive       0.61      0.84      0.71     28770

    accuracy                           0.65     56709
   macro avg       0.67      0.65      0.64     56709
weighted avg       0.67      0.65      0.64     56709



# Classification - Классификация

получаем результат анализа однограммы - we obtain the result of the unigram analysis

ValueError: X has 115446 features, but LogisticRegression is expecting 1328872 features as input.

# Биграми - Bigrami

In [ ]:
#we initial the vectorization
vectorizer_2 = CountVectorizer(ngram_range=(2,2))

vectorizer_x_train_2 = vectorizer_2.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(vectorizer_x_train_2, y_train)
vectorized_x_test_2 = vectorizer_2.transform(x_test)

pred = clf.predict(vectorized_x_test_2)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

    negative       0.73      0.69      0.71     28217
    positive       0.71      0.74      0.72     28492

    accuracy                           0.71     56709
   macro avg       0.72      0.71      0.71     56709
weighted avg       0.72      0.71      0.71     56709



# Триграммы

In [ ]:
# инициализируем векторайзер
vectorizer_3 = CountVectorizer(ngram_range=(3,3))
# обучаем его и сразу применяем к x_train
vectorized_x_train_3 = vectorizer_3.fit_transform(x_train)
# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(vectorized_x_train_3, y_train)
# применяем обученный векторизатор к тестовым данным
vectorized_x_test_3 = vectorizer_3.transform(x_test)
# получаем предсказания и выводим информацию о качестве
pred = clf.predict(vectorized_x_test_3)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.72      0.46      0.56     28217
    positive       0.61      0.82      0.70     28492

    accuracy                           0.64     56709
   macro avg       0.66      0.64      0.63     56709
weighted avg       0.66      0.64      0.63     56709



# LogistRegresion Pentagram

In [31]:
# CountVectorizer pentagramas
vectorizer_5 = CountVectorizer(ngram_range=(5,5))
X_train_5 = vectorizer_5.fit_transform(x_train)
X_test_5 = vectorizer_5.transform(x_test)

# LogisticRegression con pentagramas
clf_5 = LogisticRegression(random_state=42, max_iter=1000)
clf_5.fit(X_train_5, y_train)
pred_5 = clf_5.predict(X_test_5)
print("Pentagramas:", classification_report(y_test, pred_5))

Pentagramas:               precision    recall  f1-score   support

    negative       0.52      1.00      0.68     27961
    positive       0.96      0.11      0.19     28748

    accuracy                           0.54     56709
   macro avg       0.74      0.55      0.44     56709
weighted avg       0.74      0.54      0.43     56709



Clasification with XGBClassifier

In [33]:
!pip3 install xgboost

In [34]:
import sys
import subprocess

# Instalar xgboost
subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])
from xgboost import XGBClassifier

In [35]:


xgb_model = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=3,
    gamma=0.2,
    subsample=0.6,
    colsample_bytree=1.0,
    objective='binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27
)

Обучание - we train

In [36]:
from sklearn.preprocessing import LabelEncoder

# кодирует метки в числах - encode the labels in numbers
le = LabelEncoder()

y_train_num = le.fit_transform(y_train) # negative =0 positive = 1
y_test_num  = le.transform(y_test) #aplicar la misma transformacion

#train
xgb_model.fit(vectorized_x_train, y_train_num)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,1.0
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


Prediction

In [37]:
vectorized_x_test_XGB = vectorizer.transform(x_test)  

xgb_pred_num = xgb_model.predict(vectorized_x_test_XGB)
xgb_pred_text = le.inverse_transform(xgb_pred_num)


In [38]:
print("\n" + "="*60)
print("📊 RESULTADOS XGBClassifier")
print("="*60)
print(classification_report(y_test, xgb_pred_text))


📊 RESULTADOS XGBClassifier
              precision    recall  f1-score   support

    negative       0.74      0.68      0.71     27961
    positive       0.71      0.77      0.74     28748

    accuracy                           0.73     56709
   macro avg       0.73      0.73      0.73     56709
weighted avg       0.73      0.73      0.73     56709



# Clasificate with K-ближайших соседей (KNN) unigrama

In [ ]:
#  K-ближайших соседей (KNN) \  K-Nearest Neighbors (KNN)

from sklearn.neighbors import KNeighborsClassifier

# Создаем модель / Create model
knn = KNeighborsClassifier(
    n_neighbors=3,     # Количество соседей / Number of neighbors
    weights='distance', # Веса по расстоянию / Distance weighting
    metric='euclidean'  # Мера расстояния / Distance metric
)


# Обучение / Training
knn.fit(vectorized_x_train, y_train)

knn_pred = knn.predict(vectorized_x_test)

# Отчет / Report
print("\n=== KNN / K-ближайших соседей ===")
print(classification_report(y_test, knn_pred, digits=4))

In [ ]:
print("=============\n TF-ID УНИГРАМ - LogistRegresion ================")
print(classification_report(y_test, pred))

print("=============\n TF-ID ВЫНИГРАМ - LogistRegresion ================")
print(classification_report(y_test, pred2))

print("=============\n TF-ID Триграмма - LogistRegresion ================")
print(classification_report(y_test, pred3))

print("=============\n TF-ID 5 Gram - LogistRegresion ================")
print("Pentagramas:", classification_report(y_test, pred_5))

print("=============\n  RESULTADOS XGBClassifier ================")
print(classification_report(y_test, xgb_pred_text))

print("\n=== KNN / K-ближайших соседей ===")
print(classification_report(y_test, knn_pred, digits=4))

 TF-ID УНИГРАМ - LogistRegresion ================
              precision    recall  f1-score   support

    negative       0.73      0.78      0.75     27961
    positive       0.77      0.72      0.74     28748

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709

 TF-ID ВЫНИГРАМ - LogistRegresion ================
              precision    recall  f1-score   support

    negative       0.73      0.66      0.69     27961
    positive       0.70      0.76      0.73     28748

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709

 TF-ID Триграмма - LogistRegresion ================
              precision    recall  f1-score   support

    negative       0.73      0.46      0.56     27961
    positive       0.61      0.83      0.71     28748

    accuracy                 

NameError: name 'xgb_pred_text' is not defined